In [1]:
import cv2

In [2]:
# Load the cascade
face_cascade = cv2.CascadeClassifier('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\haarcascade_frontalface_default.xml')

In [3]:
# Read the input image
img = cv2.imread('.\\FinalTestHenon\\test.png')
img2 = cv2.imread('.\\FinalTestHenon\\test.png')

In [4]:
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

In [6]:
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img2, (x, y), (x+w, y+h), (255, 0, 0), 2)

In [7]:
# Display the output
cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestHenon\\face detection result.png', img2)

True

In [8]:
from PIL import Image
import numpy as np
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2 
import random
from math import log
from tqdm import tqdm

In [9]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
      color = 0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [10]:
def dec(bitSequence):
    decimal = 0
    for bit in bitSequence:
        decimal = decimal * 2 + int(bit)
    return decimal

In [11]:
def genHenonMap(dimension, key):
    x = key[0]
    y = key[1]
    sequenceSize = dimension * dimension * 8 #Total Number of bitSequence produced
    bitSequence = []    #Each bitSequence contains 8 bits
    byteArray = []      #Each byteArray contains m( i.e 512 in this case) bitSequence
    TImageMatrix = []   #Each TImageMatrix contains m*n byteArray( i.e 512 byteArray in this case)
    for i in range(sequenceSize):
        xN = y + 1 - 1.4 * x**2
        yN = 0.3 * x

        x = xN
        y = yN

        if xN <= 0.4:
            bit = 0
        else:
            bit = 1

        try:
            bitSequence.append(bit)
        except:
            bitSequence = [bit]

        if i % 8 == 7:
            decimal = dec(bitSequence)
            try:
                byteArray.append(decimal)
            except:
                byteArray = [decimal]
            bitSequence = []

        byteArraySize = dimension*8
        if i % byteArraySize == byteArraySize-1:
            try:
                TImageMatrix.append(byteArray)
            except:
                TImageMatrix = [byteArray]
            byteArray = []
    return TImageMatrix

### Henon Encryption

In [12]:
def HenonEncryption(imageName,key):
    imageMatrix, dimension, color = getImageMatrix(imageName)
    transformationMatrix = genHenonMap(dimension, key)
    resultantMatrix = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            try:
                if color:
                    row.append(tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]))
                else:
                    row.append(transformationMatrix[i][j] ^ imageMatrix[i][j])
            except:
                if color:
                    row = [tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]])]
                else :
                    row = [transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]
        try:    
            resultantMatrix.append(row)
        except:
            resultantMatrix = [row]
    if color:
      im = Image.new("RGB", (dimension, dimension))
    else: 
      im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels

    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = resultantMatrix[x][y]
            
    im.save(imageName + "_HenonEnc.png", "PNG")

In [13]:
#input params for arnold cat encryption function
image = ".\\FinalTestHenon\\temp"
ext = ".png"
key = (0.1,0.1)

In [14]:
for (x, y, w, h) in faces:
    cropped_image = img[y:y+h, x:x+w]
    cv2.imshow("crop faces", cropped_image)
    cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestHenon\\temp.png', cropped_image)
    HenonEncryption(image + ext, key)
    im = Image.open(image + ".png_HenonEnc.PNG", 'r')
    #imshow(np.asarray(im))
    HenonResult = cv2.imread(".\\FinalTestHenon\\temp.png_HenonEnc.PNG",cv2.IMREAD_COLOR)
    #print(HenonResult)
    cv2.imshow('Henon Result', HenonResult)
    img[y:y+h, x:x+w] = HenonResult
    #cv2.waitKey(0)

In [15]:
cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestHenon\\Encryption Result.png', img)

True

### Henon Decryption

In [49]:
def HenonDecryption(imageNameEnc, key):
    imageMatrix, dimension, color = getImageMatrix(imageNameEnc)
    transformationMatrix = genHenonMap(dimension, key)
    pil_im = Image.open(imageNameEnc, 'r')
    imshow(np.asarray(pil_im))
    henonDecryptedImage = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            try:
                if color:
                    row.append(tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]))
                else:
                    row.append(transformationMatrix[i][j] ^ imageMatrix[i][j])
            except:
                if color:
                    row = [tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]])]
                else :
                    row = [transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]
        try:
            henonDecryptedImage.append(row)
        except:
            henonDecryptedImage = [row]
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels

    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = henonDecryptedImage[x][y]
    im.save(imageNameEnc.split('_')[0] + "_HenonDec.png", "PNG")

In [50]:
#decryption test
imgd = cv2.imread('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\decrypt.jpg')

for (x, y, w, h) in faces:
    cropped_image = imgd[y:y+h, x:x+w]
    cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\test.jpg', cropped_image)
    c_image=cv2.imread('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\test.jpg')
    ArnoldCatDecrypt = ArnoldCatDecryption(image + ext , key)
    imgd[y:y+h, x:x+w] = ArnoldCatDecrypt
    cv2.waitKey(0)

TypeError: 'NoneType' object is not subscriptable